In [1]:
from flask import Flask, request, jsonify
from flask_pymongo import PyMongo
from bson.objectid import ObjectId
from pymongo import MongoClient

# 預處理

In [ ]:
## 有一個在docker裡的mongoDB
## docker pull mongo
## docker run --name mongo1 -v C:\Users\PY.Chih\Desktop\AIOT\D18_flask-pymongo\data:/data/db -d -p 8088:27017 mongo:latest

# 建立database

In [ ]:
client = MongoClient(host='127.0.0.1', port=8088)
db = client['person_data']
collection = db['d0206']
data_list = [
    {'name' : 'Kitty', 'age': 18, 'gender': 1, 'emotion':'happy'},
    {'name' : 'Michael', 'age': 3, 'gender': 0, 'emotion': 'angry'},
    {'name' : 'Jack', 'age': 16, 'gender': 1, 'emotion': 'disgust'},
    {'name' : 'Harry', 'age': 60, 'gender': 0, 'emotion': 'fear'},
    {'name' : 'David', 'age': 18, 'gender': 1, 'emotion': 'neutral'},
    {'name' : 'Thomas', 'age': 33, 'gender': 1, 'emotion': 'sad'},
    {'name' : 'George', 'age': 54, 'gender': 0, 'emotion': 'surprise'},
    {'name' : 'William', 'age': 26, 'gender': 0, 'emotion': 'neutral'},
    {'name' : 'Charles', 'age': 80, 'gender': 1, 'emotion': 'happy'}
]
result = collection.insert_many(data_list)
## 用container裡的mongo shell 確認資料有新增
## docker exec -it mongo1 mongo
## show dbs
## show collections
## db.d0206.find()

# 使用 Flask 操作 MongoDB - 取得

In [ ]:
app = Flask(__name__)
app.config["MONGO_URI"] = "mongodb://localhost:8088/person_data"
mongo = PyMongo(app)

@app.route('/d0206')
@app.route('/d0206/<id>', methods = ['GET'])
def get_member(id=None):
    # 若沒有傳入 id，及取得所有資料
    if id is None:
        members = mongo.db.d0206.find({})
        result = [] # 宣告一個陣列，回傳結果時需要用到
        
        for member in members:
            member['_id'] = str(member['_id']) # mongo 回傳的 id 為 ObjectId 型別，把 id 轉成一般的字串
            result.append(member)
        return jsonify(result) # 把我們的結果轉換為 json 並回傳
    else:
        result = mongo.db.d0206.find_one({'_id': ObjectId(id)})
        if result is not None:
            result['_id'] = str(result['_id']) # mongo 回傳的 id 為 ObjectId 型別，把 id 轉成一般的字串
        
        return jsonify(result) # 把我們的結果轉換為 json 並回傳
if __name__ == '__main__':
    app.run()
## 所有資料的可以
## id的找不到

# 使用 Flask 操作 MongoDB - 新增

In [ ]:
app = Flask(__name__)
app.config["MONGO_URI"] = "mongodb://localhost:8088/person_data"
mongo = PyMongo(app)

@app.route('/d0206', methods = ['POST'])
def add_member(id=None):
    name = request.form.get('name')
    age = request.form.get('age')
    gender = request.form.get('gender')
    emotion = request.form.get('emotion')
    result = mongo.db.d0206.insert_one({'name': name, 'age': age, 'gender': gender, 'emotion': emotion})
    return str(result.inserted_id)

if __name__ == '__main__':
    app.run()

# 使用 Flask 操作 MongoDB - 刪除

In [ ]:
app = Flask(__name__)
app.config["MONGO_URI"] = "mongodb://localhost:8088/person_data"
mongo = PyMongo(app)

@app.route('/d0206/<id>', methods = ['DELETE'])
def remove_member(id):
    result = 0
    member = mongo.db.d0206.find_one({'_id': ObjectId(id)})
    if member is not None:
        del_result = mongo.db.d0206.delete_one(member)
        result = del_result.deleted_count
    return "Delete {} data".format(result)
if __name__ == '__main__':
    app.run()

In [3]:
app = Flask(__name__)
app.config["MONGO_URI"] = "mongodb://localhost:8088/person_data"
mongo = PyMongo(app)

@app.route('/d0206/<id>', methods = ['PUT'])
def update_member(id):
    result = 0
    
    name = request.form.get('name')
    age = request.form.get('age')
    gender = request.form.get('gender')
    emotion = request.form.get('emotion')
    new_value = { "$set": { "name":name, "age": age, "gender":gender, "emotion": emotion}}
    upd_result = mongo.db.d0206.update_one({"_id": ObjectId(id)}, new_value)
    if upd_result is not None:
        result = upd_result.modified_count
    return "update {} data".format(result)

if __name__ == "__main__":
    app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [26/Feb/2021 19:34:34] "PUT /d0206/6038d47db85e51bf14d638ca HTTP/1.1" 200 -
